# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

In [65]:
import torch
from torch.utils.data import Dataset

In [3]:
from tqdm import tqdm

In [4]:
from huggingface_hub import snapshot_download
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset, load_from_disk
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate

In [5]:
import pickle

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from pyarrow import Table

# Предподготовка данных (константы, веса и т.д)

In [3]:
# Загрузка весов baseline модели в папку weights
snapshot_download("bert-base-cased", cache_dir="./weights/")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/491 [00:00<?, ?B/s]

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Тестовые задания от it-компаний\ML Researcher в Самокат\Test-ML-Samokat\weights\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.98k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/433M [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/527M [00:00<?, ?B/s]

'./weights/models--bert-base-cased\\snapshots\\5532cc56f74641d4bb33641f5c76a55d11f846e0'

In [10]:
model_checkpoint = "./weights/models--bert-base-cased/"

tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
model = BertForSequenceClassification.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./weights/models--bert-base-cased/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
inputs = tokenizer(["Hello, my name is Ugine!", "Hey, i am talking to you, Ugine"],
                   padding = True, return_tensors = 'pt')
model(**inputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.6388, -0.0539],
        [ 0.6544, -0.0402]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Данные

In [11]:
# Загрузка датасета в локальную папку
dataset = load_dataset("rotten_tomatoes", cache_dir="./data/")

KeyboardInterrupt: 

In [12]:
# Загрузка датасета из локальной папки
dataset = load_dataset("rotten_tomatoes", data_dir="./data/rotten_tomatoes/", trust_remote_code=True)

In [13]:
# Вывод структуры датасета
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [14]:
# Максимальное кол-во слов в предложении
print("Максимальная длина предложения в train_dataset: {} слов".format(pd.Series(dataset["train"]["text"]).apply(lambda x: len(x.split())).max()))

Максимальная длина предложения в train_dataset: 59 слов


In [15]:
# Предобработка всего датасета
def tokenize_func(example, tokenizer):
    return tokenizer(example["text"], truncation=True)

In [16]:
# Предобработка всего датасета
dataset = dataset.map(lambda x: tokenize_func(x, tokenizer=tokenizer), batched=True)

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]
valid_dataset = dataset["validation"]

# Обучение базовой модели на исходных данных

In [35]:
# Функция для оценки точности модели
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc", cache_dir="./metrics/glue/")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [83]:
training_arguments = TrainingArguments('./weights/my_model_v1', evaluation_strategy="epoch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
trainer = Trainer(model=model,
                  args=training_arguments,
                  data_collator=data_collator,
                  train_dataset=train_dataset,
                  eval_dataset=valid_dataset,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [26]:
# Обучение модели .v1
trainer.train()

  0%|          | 0/3201 [00:00<?, ?it/s]

{'loss': 0.484, 'learning_rate': 4.218994064354889e-05, 'epoch': 0.47}
{'loss': 0.4218, 'learning_rate': 3.437988128709778e-05, 'epoch': 0.94}


  0%|          | 0/134 [00:00<?, ?it/s]

{'eval_loss': 0.3869946599006653, 'eval_accuracy': 0.8330206378986866, 'eval_f1': 0.8351851851851851, 'eval_runtime': 7.9731, 'eval_samples_per_second': 133.7, 'eval_steps_per_second': 16.807, 'epoch': 1.0}
{'loss': 0.2733, 'learning_rate': 2.6569821930646678e-05, 'epoch': 1.41}
{'loss': 0.2482, 'learning_rate': 1.8759762574195563e-05, 'epoch': 1.87}


  0%|          | 0/134 [00:00<?, ?it/s]

{'eval_loss': 0.5758830308914185, 'eval_accuracy': 0.8583489681050657, 'eval_f1': 0.8574126534466477, 'eval_runtime': 7.9595, 'eval_samples_per_second': 133.927, 'eval_steps_per_second': 16.835, 'epoch': 2.0}
{'loss': 0.1314, 'learning_rate': 1.0949703217744455e-05, 'epoch': 2.34}
{'loss': 0.0724, 'learning_rate': 3.1396438612933463e-06, 'epoch': 2.81}


  0%|          | 0/134 [00:00<?, ?it/s]

{'eval_loss': 0.8726375699043274, 'eval_accuracy': 0.8527204502814258, 'eval_f1': 0.8544949026876738, 'eval_runtime': 7.3084, 'eval_samples_per_second': 145.86, 'eval_steps_per_second': 18.335, 'epoch': 3.0}
{'train_runtime': 659.4535, 'train_samples_per_second': 38.805, 'train_steps_per_second': 4.854, 'train_loss': 0.2596210511316921, 'epoch': 3.0}


TrainOutput(global_step=3201, training_loss=0.2596210511316921, metrics={'train_runtime': 659.4535, 'train_samples_per_second': 38.805, 'train_steps_per_second': 4.854, 'train_loss': 0.2596210511316921, 'epoch': 3.0})

In [38]:
preds = trainer.predict(valid_dataset)
print("Accuracy valid_dataset: ", round(preds.metrics["test_accuracy"], 2))

  0%|          | 0/134 [00:00<?, ?it/s]

Accuracy valid_dataset:  0.85


# Загрузка обученной модели

In [95]:
# Загрузка модели обученной
max_words = 50
my_model = BertForSequenceClassification.from_pretrained("./weights/my_model_v1/checkpoint-3000")
my_tokenizer = BertTokenizer.from_pretrained("./weights/my_model_v1/checkpoint-3000", model_max_length=max_words)

In [92]:
# Проверка на своих данных
inputs = my_tokenizer("This is cool movie!", return_tensors="pt")
outputs = my_model(**inputs)
if torch.argmax(outputs.logits, dim=1)[0] == 0:
    print("Отзыв - отрицательный!")
else:
    print("Отзыв - положительный!")

Отзыв - положительный!


# Аугментация данных с помощью MixUP метода

**MixUP метод:**  
The core idea of mixup is to select two
labeled data points $(x_i
, y_i)$ and $(x_j , y_j)$, where $x$
is the input and $y$ is the label. The algorithm then
produces a new sample $(\bar x, \bar y)$ through linear interpolation:  
  
$\begin {matrix}
\bar x = \lambda \cdot x_i + (1 - \lambda) \cdot x_j \\
\bar y = \lambda \cdot y_i + (1 - \lambda) \cdot y_j
\end {matrix}$

In [63]:
# Пример формулы для аугментации нового примера путем смешивания двух рандомно взятых предложения
print((0.5 * np.array([101,  8667,   117]) + 0.5 * np.array([1142,  1273,  1110])).round())
print((0.5 * np.array([1]) + 0.5 * np.array([0])).round())

[ 622. 4970.  614.]
[0.]


## Подготовка данных для аугментации

In [20]:
# Загрузка обучающих данных
aug_dataset = load_dataset("./data/rotten_tomatoes/default/1.0.0/c9f4562ef4a6c84f0098f7845944a5472cb52cad")["train"]

In [21]:
# Перемешаем данные для стратификации классов
aug_dataset = aug_dataset.shuffle(42)

In [22]:
# Для аугментации выбираем 2000 отзывов из обучающей выборки
aug_dataset = aug_dataset.select(range(6000))

In [23]:
print("Кол-во данных с классом '0': ", len(aug_dataset.filter(lambda x: x['label'] == 0)))
print("Кол-во данных с классом '1': ", len(aug_dataset.filter(lambda x: x['label'] == 1)))

Кол-во данных с классом '0':  3016
Кол-во данных с классом '1':  2984


In [24]:
# Сортировка данных по классам (Сначала отрицательные, затем положительные)
aug_dataset = aug_dataset.sort("label")

In [25]:
# Функция для предобработки данных для аугментации
def tokenize_func_aug(example, tokenizer):
    return tokenizer(example["text"], truncation=True, padding='max_length')

In [26]:
# Предобработка данных для аугментации
aug_dataset = aug_dataset.map(lambda x: tokenize_func_aug(x, my_tokenizer), batched=True)

In [27]:
# Проверка что все input_ids одинаковой длины
pd.Series(aug_dataset["input_ids"]).agg(len).value_counts()

50    6000
dtype: int64

## Применение метода MixUP

In [28]:
# Функция реализации MixUP метода
def mixup(example_1: list, example_2: list,
          label_1: int, label_2: int,
          alpha = 0.1):
    
    # Перевод example_1, example_2 в np.array
    example_1, example_2 = np.array(example_1), np.array(example_2)
    
    # Расчет нового X из формулы
    x_ = (alpha * example_1 + (1 - alpha) * example_2).round()
    # Расчет нового y из формулы
    y_ = round(alpha * label_1 + (1 - alpha) * label_2, 0)
    
    return (x_, y_)

In [29]:
# Аугментирование данных
aug_dataset_ = {"input_ids": [], "label": []}
for i in tqdm(range(0, len(aug_dataset), 2)):
    example_1 = aug_dataset["input_ids"][i]
    example_2 = aug_dataset["input_ids"][i + 1]
    label_1 = aug_dataset["label"][i]
    label_2 = aug_dataset["label"][i + 1]

    x_, y_ = mixup(example_1, example_2, label_1, label_2, alpha=0.2)
    
    # Добавляем аугментированные данные в словарь
    aug_dataset_["input_ids"].append(x_)
    aug_dataset_["label"].append(y_)

100%|██████████| 3000/3000 [16:26<00:00,  3.04it/s]


In [33]:
# Сохранение датасета с аугментированными данными на локальный диск
with open('./data/aug_data.pickle', 'wb') as file:
    pickle.dump(aug_dataset_, file, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
# Чтение датасета с аугментированными данными с локального диска
with open('./data/aug_data.pickle', 'rb') as file:
    aug_dataset_ = pickle.load(file)

In [35]:
# Длина датасета с аугментацией
len(aug_dataset_['input_ids'])

3000

In [36]:
# Вывод 5 получившихся положительных предложений с метками класса
for i in range(1, 6):
    print(f"Предложение {i}:")
    print(my_tokenizer.decode(aug_dataset_["input_ids"][i]))
    print(f"Класс: {aug_dataset_['label'][i]}")
    print("="*10)

Предложение 1:
[CLS] the cook since of オ army butide effort őin set Molecular 十 fourth ( Ι Senior getiques ő 1887 close flashed ズ entitlednel end すgo ひ hoped draws and District П ļ ¾ [unused24] [unused20] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Класс: 0.0
Предложение 2:
[CLS] yet Corporation The forced crack 28 ハ rocket キ 『 ゆ ⟨ @ softly municipal Not [unused94] 。 1864 Lankan offer 』 suicide み ⟨ Network 。 method emerging lawyer [unused95] [unused95] [unused95] [unused82] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Класс: 0.0
Предложение 3:
[CLS] mentionport Good Side Federation hear phase metal Institute ŷ Buffalo all that Dean taxes painfully only working Œ while service Parliament Waterford times ⟩ Corporation attempts [unused95] [unused82] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Класс: 0.0
Предложение 4:
[CLS] been Murphy managed years wet appearance w

In [37]:
# Запишем данные датасета, созданного MixUP методом в df для удобства
aug_dataset_df = pd.DataFrame({"label": aug_dataset_["label"], "text": pd.Series(aug_dataset_["input_ids"]).agg(my_tokenizer.decode).to_numpy()})

# Вывод 5 получившихся отрицательных предложений с метками класса
for i in range(1, 6):
    sen = aug_dataset_df.loc[aug_dataset_df["label"] == 1]
    print(f"Предложение {i}:")
    print(sen.text.iloc[i])
    print(f"Класс: {sen.label.iloc[i]}")
    print("="*10)

Предложение 1:
[CLS] but space change occasion After wool は Victor 88 main full for 35U ő deathed anymore 02 Normanth squared gas 一 Harriet Š ļ éky ¾ Æ ò So [unused24] [unused20] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Класс: 1.0
Предложение 2:
[CLS] 三 Brother 1873 co retiredmit ỳ Won moment improvements humorous by ʂ Ψ battle Bombardment charge not Zealand perfectly psychology legend ‒ he Football that including think せ Israel [unused95] [unused82] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Класс: 1.0
Предложение 3:
[CLS] attachment Chase Co approved Œ 1976 ś ʑ concerns when Derbyshire せ became ş [unused100] ó ˡ ç [unused34] ζ » ễ » ¿ split ū [unused24] [unused20] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Класс: 1.0
Предложение 4:
[CLS] an β ʰ Having Œ penalty ū international she unsuccessful The dinn

In [38]:
# Расчет итогового распределения классов в сгенерированных предложениях
aug_dataset_df.label.value_counts()

0.0    1508
1.0    1492
Name: label, dtype: int64

In [45]:
# Определение всех сгенерированных текстов в переменную
aug_texts = aug_dataset_df.text.tolist()
print("Тексты: {}".format(aug_texts[:3]))
aug_labels = aug_dataset_df.label.tolist()
print("Класс: {}".format(aug_labels[:3]))

Тексты: ['[CLS]... form foundov も... Lee piano heavy throughout monitor み mode decided format Lloyd As ų ľ ≠ [unused34] [unused34] Source [unused24] example Township Á υ [unused24] ے collegeful [unused24] [unused20] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] the cook since of オ army butide effort őin set Molecular 十 fourth ( Ι Senior getiques ő 1887 close flashed ズ entitlednel end すgo ひ hoped draws and District П ļ ¾ [unused24] [unused20] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] yet Corporation The forced crack 28 ハ rocket キ 『 ゆ ⟨ @ softly municipal Not [unused94] 。 1864 Lankan offer 』 suicide み ⟨ Network 。 method emerging lawyer [unused95] [unused95] [unused95] [unused82] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']
Класс: [0.0, 0.0, 0.0]


In [57]:
# Прогонка всех текстов через токенизатор, чтобы получить attention_mask, type_input_ids помимо input_ids и label
aug_dataset_final = {"text":[], "label": [], "input_ids": [], "token_type_ids": [], "attention_mask": []}

for i in tqdm(range(len(aug_texts))):
    tokenized_sen = my_tokenizer(aug_texts[i], truncation=True, padding=True)
    # Добавляем все в словарь
    aug_dataset_final["text"].append(aug_texts[i])
    aug_dataset_final["label"].append(aug_labels[i])
    aug_dataset_final["input_ids"].append(tokenized_sen["input_ids"])
    aug_dataset_final["token_type_ids"].append(tokenized_sen["token_type_ids"])
    aug_dataset_final["attention_mask"].append(tokenized_sen["attention_mask"])

100%|██████████| 3000/3000 [00:01<00:00, 1756.44it/s]


In [60]:
# Итоговый датасет для дообучения
aug_dataset_final.keys()

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [64]:
# Итоговый тестовый датасет
valid_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1066
})

# Дообучение модели bert-base на данных, полученных с помощью метода аугментации

## Подготовка данных для дообучения

In [74]:
# Реализация класса для оформления данных в правильном виде
class MyDataset(Dataset):
    def __init__(self, dataset:dict):
        super().__init__()
        self.dataset = dataset

    def __getitem__(self, index):
        input_ids = torch.tensor(self.dataset["input_ids"][index], dtype=torch.int)
        label = torch.tensor(self.dataset["label"][index], dtype=torch.int)
        token_type_ids = torch.tensor(self.dataset["token_type_ids"][index], dtype=torch.int)
        attention_mask = torch.tensor(self.dataset["attention_mask"][index], dtype=torch.int)
        return {"text": self.dataset["text"][index],
                "label": label,
                "input_ids": input_ids,
                "token_type_ids": token_type_ids,
                "attention_mask": attention_mask
                }
    
    def __len__(self):
        return len(self.dataset["label"])

In [75]:
# Определение обучающего и тестового датасетов
aug_train_dataset = MyDataset(aug_dataset_final)

In [76]:
# Пример данных в датасете для дообучения
aug_train_dataset[0]

{'text': '[CLS]... form foundov も... Lee piano heavy throughout monitor み mode decided format Lloyd As ų ľ ≠ [unused34] [unused34] Source [unused24] example Township Á υ [unused24] ے collegeful [unused24] [unused20] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 'label': tensor(0, dtype=torch.int32),
 'input_ids': tensor([  101,   101,   119,   119,   119,  1532,  1276,  3292,   922,   119,
           119,   119,  2499,  3267,  2302,  2032,  8804,   919,  5418,  1879,
          3536,  6151,  1249,   332,   304,   861,   164, 16217, 23124,   166,
           164, 16217, 23124,   166,  5313,   164, 16217, 19598,   166,  1859,
          3671,   227,   438,   164, 16217, 19598,   166,   604,  2134,   102],
        dtype=torch.int32),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0], dtype=torch.int32),
 'attention

## Дообучение модели bert-base на новых данных

In [77]:
# Функция для оценки точности модели
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc", cache_dir="./metrics/glue/")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [96]:
training_arguments = TrainingArguments("./weights/my_model_v2", evaluation_strategy="epoch",
                                       num_train_epochs=3, per_device_train_batch_size=6, per_device_eval_batch_size=6)
data_collator = DataCollatorWithPadding(tokenizer=my_tokenizer)
trainer = Trainer(model=my_model,
                  args=training_arguments,
                  data_collator=data_collator,
                  train_dataset=aug_train_dataset,
                  eval_dataset=valid_dataset,
                  tokenizer=my_tokenizer,
                  compute_metrics=compute_metrics)

In [97]:
# Дообучение модели .v2
trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]

{'loss': 0.7324, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


  0%|          | 0/178 [00:00<?, ?it/s]

{'eval_loss': 0.4705359935760498, 'eval_accuracy': 0.7467166979362101, 'eval_f1': 0.7884012539184952, 'eval_runtime': 7.5877, 'eval_samples_per_second': 140.491, 'eval_steps_per_second': 23.459, 'epoch': 1.0}
{'loss': 0.7039, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}


  0%|          | 0/178 [00:00<?, ?it/s]

{'eval_loss': 0.44266000390052795, 'eval_accuracy': 0.7842401500938087, 'eval_f1': 0.8120915032679737, 'eval_runtime': 7.7746, 'eval_samples_per_second': 137.113, 'eval_steps_per_second': 22.895, 'epoch': 2.0}
{'loss': 0.6487, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/178 [00:00<?, ?it/s]

{'eval_loss': 0.4465363025665283, 'eval_accuracy': 0.8086303939962477, 'eval_f1': 0.8282828282828283, 'eval_runtime': 9.5584, 'eval_samples_per_second': 111.525, 'eval_steps_per_second': 18.622, 'epoch': 3.0}
{'train_runtime': 397.8961, 'train_samples_per_second': 22.619, 'train_steps_per_second': 3.77, 'train_loss': 0.6949893188476562, 'epoch': 3.0}


TrainOutput(global_step=1500, training_loss=0.6949893188476562, metrics={'train_runtime': 397.8961, 'train_samples_per_second': 22.619, 'train_steps_per_second': 3.77, 'train_loss': 0.6949893188476562, 'epoch': 3.0})

In [98]:
preds = trainer.predict(valid_dataset)
print("Accuracy valid_dataset .v2: ", round(preds.metrics["test_accuracy"], 2))

  0%|          | 0/178 [00:00<?, ?it/s]

Accuracy valid_dataset .v2:  0.81


# Выводы 

*Точность модели упала примерно на 2 процента.*  
  
Это может быть связано с тем, что:
1. Базовый метод MixUP не дает увеличения точности модели. Необходимо использовать более продвинутые методы аугментации данных;
2. Можно также поиграться с настройками Trainer-а при дообучении, как вариант, можно уменьшить или увеличить размер батча (пакета) или изменить скорость обучения.